In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl
import sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
import warnings
warnings.filterwarnings("ignore")
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split


In [ ]:
#importing file 
aba=pd.read_csv("Abalone.csv")

In [ ]:
aba

Observation- This is a classification problem 

In [ ]:
aba.shape

Observation - ds has 4177 rows adn 9 columns 

Obseevation-   target value is in the end position  in the dataframe . 

In [ ]:
sns.heatmap(aba.isnull(),cmap="Spectral_r")

In [ ]:
aba.isnull().sum()

Observation - ds has no missing values- no treatment required 

In [ ]:
aba.info ()

Observation -  sex is object , needs encoding 

In [ ]:
aba.describe ()

Observation- q3 and  max in height , Whole weight,Shucked weight,	Shell weight have huge difference in beween -  indicating towards presence of outlier 

In [ ]:
aba["Height"].plot.box()


Observation - very few outliers are there  but are much higher by size

In [ ]:
aba["Whole weight"].plot.box()

Observation - some outliers are there  but are not much higher by size

In [ ]:
aba["Shucked weight"].plot.box()

Observation - some outliers are there  but are not much higher by size

In [ ]:
aba["Shell weight"].plot.box()

Observation - some outliers are there  but are not much higher by size

In [ ]:

aba.plot(kind="box",subplots=True,figsize=(15,5))

In [ ]:
aba.skew()

Observation -skewness in height is too high 
-probably will get fixed after removing the outliers . 

In [ ]:
cor=aba.corr()


In [ ]:
#un-encoded data 
aba["Sex"].value_counts()

In [ ]:
le=LabelEncoder()
aba["Sex"]=le.fit_transform(aba["Sex"].astype(str))

In [ ]:
from scipy.stats import zscore
z=np.abs(zscore(aba))
z

In [ ]:
abalone=aba[(z<3).all(axis=1)]

In [ ]:
abalone["Sex"].value_counts()

In [ ]:
abalone

In [ ]:
abalone.shape 

In [ ]:
aba.shape

In [ ]:
4027/4177

Observation-  Lost 3.6  percent data  in  removing outlir.  can be accepted . 

In [ ]:
abalone.skew()

In [ ]:
#skewness  is under control after removing  outliers  . 

In [ ]:
abalone.plot( kind="box",subplots=True , figsize=(15,4))

observation- On the above  viz outliers are present butthose are for the newly shaped dataframe so we wont try to treat these outliers again  . 

In [ ]:
sns.countplot(x="Sex", data=abalone)

In [ ]:
sns.countplot(x="Rings", data=abalone)

Observation - 9-10 rings are the major contributor to the data set  

In [ ]:
#unique values 
abalone.nunique()

In [ ]:
#encoded data 
abalone["Sex"].value_counts()

In [ ]:
abalone["Rings"].value_counts()

In [ ]:
cor2=abalone.corr()

plt.figure(figsize=(10,7))
sns.heatmap(cor2)

observation-  sex column  reacts quite negatively  after  encoding so . lets remove  that one 

In [ ]:
abalone.drop("Sex",axis=1,inplace=True)

In [ ]:
cor3=abalone.corr()

In [ ]:

plt.figure(figsize=(10,7))
sns.heatmap(cor3,annot=True)


In [ ]:
def calc_vif(ds):
    vif=pd.DataFrame()
    vif["variables"]=ds.columns
    vif["VIF FACTOR"]=[variance_inflation_factor (ds.values,i) for i in range (ds.shape[1])]
    return (vif)

In [ ]:
calc_vif(abalone)

observation- length , diameter and  whole  wieight has maximum vif so we will try to do PCA to reduce it  instead of droping . 

In [ ]:
x=abalone.drop("Rings",axis=1)
y=abalone["Rings"]

In [ ]:
x=pd.DataFrame(x)
y=pd.DataFrame(y)

In [ ]:
x.shape

In [ ]:
y.shape

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
PCa=PCA(n_components=5)

In [ ]:
x=PCa.fit_transform(x)

In [ ]:
x.shape

In [ ]:
x

In [ ]:
 x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.30,random_state=1)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
x_train=scaler.fit_transform(x_train)
x_test=scaler.transform(x_test)

In [ ]:
#saving best random state 
maxAccuracy=0
maxRS=0
for i in range (1,200):
    x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.30,random_state=i)
    from sklearn.neighbors import KNeighborsClassifier
    KNN=KNeighborsClassifier()
    KNN.fit(x_train,y_train)
    pred=KNN.predict(x_test)
    accu=accuracy_score(y_test,pred)
    if accu>maxAccuracy:
        maxAccuracy=accu
        maxRS=i

print("best accuracy  is ",maxAccuracy, "with random state ",maxRS)

In [ ]:
DTC=DecisionTreeClassifier()
DTC.fit(x_train,y_train)
pred=DTC.predict(x_test)
acc1=classification_report(y_test,pred)
print(acc1)

In [ ]:
MNB=MultinomialNB()
MNB.fit(x_train,y_train)
pred=MNB.predict(x_test)
acc2=classification_report(y_test,pred)
print(acc2)

In [ ]:
SVCc=SVC()
SVCc.fit(x_train,y_train)
pred=SVCc.predict(x_test)
acc3=classification_report(y_test,pred)
print(acc3)

In [ ]:
KNN=KNeighborsClassifier()
KNN.fit(x_train,y_train)
pred=KNN.predict(x_test)
acc3=classification_report(y_test,pred)
print(acc3)

### Finding the best model with cross validation 

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
models=(DTC,KNN,SVCc)
for i in models:
    print (i , cross_val_score(i,x,y,cv=5).mean())

### using hyper parameter

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
parameters={"kernel":["linear","poly","rbf"],
            "decision_function_shape":["ovr","ovo"],
            "C":np.arange(1.0,30.0),
           "coef0":np.arange(1.0,3.0)}

In [ ]:
GCV=GridSearchCV(SVC(),parameters,refit=True,cv=3)

In [ ]:
SVCc.get_params().keys()

In [ ]:
GCV.fit(x_train,y_train)

In [ ]:
GCV.best_params_

### saving the final model 

In [ ]:
abalone_final=SVC(C=7.0, coef0=1.0, decision_function_shape= "ovr", kernel='rbf',probability=True)

In [ ]:
abalone_final.fit(x_train, y_train)
pred_ab=abalone_final.predict(x_test)
acc=accuracy_score(y_test,pred)
print (acc*100)

### Saving model 

In [ ]:
import joblib

In [ ]:
joblib.dump(abalone_final,"abalone_final.pkl")